****

In [1]:
import os
os.listdir("../input")

['genmodel', 'generative-dog-images', 'gendog']

In [ ]:
import sys
sys.path.insert(0, '../input/gendog')

In [ ]:
import train

In [2]:
from shutil import copyfile
copyfile('../input/gendog/train.py', './train.py')
copyfile('../input/gendog/parameters.py', './parameters.py')
copyfile('../input/gendog/self_atten_gan.py', './self_atten_gan.py')
copyfile('../input/gendog/layers.py', './layers.py')
copyfile('../input/gendog/data_processing.py', './data_processing.py')
copyfile('../input/gendog/utils.py', './utils.py')
copyfile('../input/gendog/generate_samples.py', './generate_samples.py')

'./utils.py'

In [3]:
os.listdir('.')

['__notebook_source__.ipynb',
 '.ipynb_checkpoints',
 'parameters.py',
 'train.py',
 'self_atten_gan.py',
 'data_processing.py',
 'layers.py',
 'utils.py']

In [4]:
os.system("python3 train.py --dataroot='../input/generative-dog-images/' --num_epoches=10")

0

In [8]:
os.listdir('../sagan_models/')

['gen_model.pth']

In [9]:
copyfile('../input/gendog/generate_samples.py', './generate_samples.py')

'./generate_samples.py'

In [10]:
os.system("python3 generate_samples.py --num_sample_images=100")

0

In [12]:
os.listdir('../output_images/')

['image_00041.png',
 'image_00015.png',
 'image_00023.png',
 'image_00094.png',
 'image_00036.png',
 'image_00084.png',
 'image_00073.png',
 'image_00046.png',
 'image_00045.png',
 'image_00096.png',
 'image_00018.png',
 'image_00048.png',
 'image_00016.png',
 'image_00035.png',
 'image_00017.png',
 'image_00087.png',
 'image_00026.png',
 'image_00075.png',
 'image_00080.png',
 'image_00098.png',
 'image_00001.png',
 'image_00089.png',
 'image_00012.png',
 'image_00063.png',
 'image_00058.png',
 'image_00031.png',
 'image_00078.png',
 'image_00029.png',
 'image_00054.png',
 'image_00057.png',
 'image_00051.png',
 'image_00030.png',
 'image_00083.png',
 'image_00009.png',
 'image_00055.png',
 'image_00021.png',
 'image_00044.png',
 'image_00008.png',
 'image_00088.png',
 'image_00040.png',
 'image_00028.png',
 'image_00003.png',
 'image_00000.png',
 'image_00002.png',
 'image_00027.png',
 'image_00077.png',
 'image_00037.png',
 'image_00092.png',
 'image_00093.png',
 'image_00056.png',


In [ ]:
import layers
import data_processing
import self_atten_gan

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Args(object):
    dataroot: str = '../input/generative-dog-images/'    
    sample_images_path: str = '../output_images/'
    submission_dir: str = './submission/'    
    save_model_dir: str = './sagan_models/'
    batch_size: int = 32
    d_lr: float = 0.0004
    g_lr: float = 0.0002
    beta1: float = 0.0
    beta2: float = 0.999
    adv_loss: str = 'dcgan'
    z_dim: int = 180
    g_conv_dim: int = 64
    d_conv_dim: int = 64
    imsize: int = 64
    num_epoches: int = 10
    real_label_value: float = 0.8
    fake_label_value: float = 0
    num_of_classes: int = 120
    num_sample_images: int = 10000
        
args = Args()

In [ ]:
gen_model, gen_losses, dis_losses = train.train_model(args)

In [ ]:
from pathlib import Path

In [ ]:
gen_model_dir = Path(args.save_model_dir)
gen_model_dir.mkdir(exist_ok=True)
torch.save(gen_model.state_dict(), os.path.join(args.save_model_dir, 'gen_model.pth'))

In [ ]:
import generate_samples
import utils

In [ ]:
os.system("python3 generate_samples.py")

In [ ]:
import torch
import self_atten_gan
import os

seed = 1234
utils.seed_everything(seed)

device = torch.device('cuda')
gen_model = self_atten_gan.Generator(args.z_dim, args.g_conv_dim, args.num_of_classes).to(device)
gen_model.load_state_dict(torch.load(os.path.join(args.save_model_dir, 'gen_model.pth')))
generate_samples.save_images(generator_model=gen_model,
                sample_images_path=args.sample_images_path,
                submission_dir=args.submission_dir,
                z_dim=args.z_dim,
                num_of_classes=args.num_of_classes,
                num_images=100)

**Import Packages**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import random

import numpy as np
import pandas as pd

import torch

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed = 1234
seed_everything(seed)

In [ ]:
train_dataset = data_processing.get_train_dataset(args.dataroot, args.imsize)

In [ ]:
num_of_classes = len(set(train_dataset.classes))
train_dataloader = data_processing.get_dataloader(train_dataset, args.batch_size, num_workers=4)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = next(iter(train_dataloader))
fig = plt.figure(figsize=(25, 16))
for ii, (img, breed_idx) in enumerate(zip(x[0][:32], x[1][:32])):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    
    img = img.numpy().transpose(1, 2, 0)
    #img_class = breeds[breed_idx]
    #plt.title(img_class)
    plt.imshow(img)

In [ ]:
device = torch.device('cuda')

### self.build_models()
G = self_atten_gan.Generator(args.z_dim, args.g_conv_dim, num_of_classes).to(device)
D = self_atten_gan.Discriminator(args.d_conv_dim, num_of_classes).to(device)

G_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, G.parameters()), args.g_lr, [args.beta1, args.beta2])
D_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), args.d_lr, [args.beta1, args.beta2])

lr_schedulerG = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(G_optimizer, T_0=args.num_epoches//10, eta_min=0.00001)
lr_schedulerD = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(D_optimizer, T_0=args.num_epoches//10, eta_min=0.00005)

In [ ]:
import torch.nn as nn
import torch.backends.cudnn as cudnn

from tqdm import tqdm_notebook as tqdm

In [ ]:
if args.adv_loss == 'dcgan':
    criterion = nn.BCELoss()
    
cudnn.benchmark = True

G.train()
D.train()

label = torch.full((args.batch_size,), 1, device=device)
ones = torch.full((args.batch_size,), 1, device=device)

gen_losses = []
dis_losses = []

# Start training
for epoch in range(args.num_epoches):
        
    for i, (real_images, dog_labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        
        real_images = real_images.to(device)
        dog_labels = torch.tensor(dog_labels, device=device)
                
        # =================================== TRAIN D ============================== #
        G_optimizer.zero_grad()
        D_optimizer.zero_grad()
        
        # TRAIN with REAL        
        # Get D output for real images & real labels
        d_out_real = D(real_images, dog_labels)
        
        
        # Compute D loss with real images & real_labels
        label.fill_(args.real_label_value) +  np.random.uniform(-0.1, 0.1)
        d_loss_real = criterion(torch.sigmoid(d_out_real), label)
        
        # Backward
        d_loss_real.backward()
        
        # Train with FAKE        
        # Create random noise
        z = torch.randn(args.batch_size, args.z_dim, device=device)
        
        # Generate fake images for same dog labels
        fake_images = G(z, dog_labels)
        
        # Get D output for fake images & same dog labels
        d_out_fake = D(fake_images.detach(), dog_labels)
        
        # Compute D loss with fake images & real labels
        label.fill_(args.fake_label_value) + np.random.uniform(0, 0.2)
        d_loss_fake = criterion(torch.sigmoid(d_out_fake), label)
        
        # Backward
        d_loss_fake.backward()
        
        d_loss = d_loss_real + d_loss_fake
        D_optimizer.step()
        
        # ====================================== TRAIN G ============================== #
        G.zero_grad()
        
        # Get D output for fake images & same dog labels
        d_out_fake = D(fake_images, dog_labels)
        
        # Compute G loss with fake images & dog_labels
        label.fill_(args.real_label_value)
        g_loss = criterion(torch.sigmoid(d_out_fake), label)
        
        g_loss.backward()
        
        G_optimizer.step()
        
        gen_losses.append(g_loss.item())
        dis_losses.append(d_loss.item())
                
        lr_schedulerG.step(epoch)
        lr_schedulerD.step(epoch)

In [ ]:
# plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(gen_losses,label="G")
plt.plot(dis_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from pathlib import Path
gen_model_dir = Path(args.save_model_dir)
gen_model_dir.mkdir(exist_ok=True)
torch.save(G.state_dict(), os.path.join(args.save_model_dir, 'gen_model.pth'))

In [ ]:
gen_model = self_atten_gan.Generator(args.z_dim, args.g_conv_dim, num_of_classes).to(device)
gen_model.load_state_dict(torch.load(os.path.join(args.save_model_dir, 'gen_model.pth')))

In [ ]:
from torchvision.utils import save_image

In [ ]:
save_images(generator_model=gen_model, 
            sample_images_path=args.sample_images_path, 
            submission_dir=args.submission_dir, 
            z_dim=args.z_dim, 
            num_images=100)

In [ ]:
os.listdir(args.sample_images_path)

In [ ]:
os.listdir(args.submission_dir)

In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import glob
import time
from dataclasses import dataclass
from pathlib import Path


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from scipy import linalg
import tensorflow as tf

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils import spectral_norm
from torch.nn.init import xavier_uniform_

import torchvision.datasets as dse
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from PIL import Image 
import xml.etree.ElementTree as ET 

from tqdm import tqdm_notebook as tqdm

**Model and Training Parameter Settings**

In [ ]:
@dataclass
class Args(object):
    dataroot: str = '../input/'    
    sample_images_path: str = '../output_images/'
    submission_dir: str = './submission/'    
    batch_size: int = 32
    d_lr: float = 0.0004
    g_lr: float = 0.0002
    beta1: float = 0.0
    beta2: float = 0.999
    adv_loss: str = 'dcgan'
    z_dim: int = 180
    g_conv_dim: int = 64
    d_conv_dim: int = 64
    imsize: int = 64
    num_epoches: int = 10
    real_label_value: float = 0.8
    fake_label_value: float = 0
        
args = Args()

**Seed Everything**

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed = 1234
seed_everything(seed)

**Data Processing**

In [ ]:
class DogDataset(Dataset):
    def __init__(self, dataroot, transform=None):
        self.images, self.classes = self.crop_images(dataroot)                
        self.transform = transform
        
    def __getitem__(self, index):
        img = self.images[index]
        img_class = self.classes[index]
        
        if self.transform is not None:
            img = self.transform(img)
        
        return (img, img_class)
    
    def __len__(self):
        return len(self.images)
    
    def crop_images(self, dataroot):
        images = []
        classes = []
        breeds = os.listdir(dataroot + 'annotation/Annotation/') 
        for breed_idx, breed in enumerate(breeds):
            for dog in os.listdir(dataroot + 'annotation/Annotation/'+ breed):
                try: 
                    img = Image.open(dataroot + 'all-dogs/all-dogs/' + dog +'.jpg') 
                except: 
                    continue           
                tree = ET.parse(dataroot + 'annotation/Annotation/'+ breed + '/'+ dog)
                root = tree.getroot()
                objects = root.findall('object')
                if len(objects) == 1:
                    bndbox = list(objects)[0].find('bndbox') 
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)
                    box_sz = np.min((xmax - xmin, ymax - ymin))
                    
                    image_w = img.size[0]
                    image_h = img.size[1]
                    image_sz = np.min((image_w,image_h))
                    if box_sz / image_sz >= 0.75:
                        a=0; b=0
                        if image_w < image_h: b = (image_h-image_sz) // 2
                        else: a = (image_w-image_sz) // 2
                        img2 = img.crop((0+a, 0+b, image_sz+a, image_sz+b))
                        img2 = img2.resize((64,64), Image.ANTIALIAS)   
                        images.append(img2)
                        classes.append(breed_idx)
                    else:
                        img2 = img.crop((xmin, ymin, xmin+box_sz, ymin+box_sz))
                        img2 = img2.resize((64,64), Image.ANTIALIAS)
                        images.append(img2)
                        classes.append(breed_idx)
                else:
                    for o in objects:
                        bndbox = o.find('bndbox') 
                        xmin = int(bndbox.find('xmin').text)
                        ymin = int(bndbox.find('ymin').text)
                        xmax = int(bndbox.find('xmax').text)
                        ymax = int(bndbox.find('ymax').text)
                        w = np.min((xmax - xmin, ymax - ymin))
                        img2 = img.crop((xmin, ymin, xmin+w, ymin+w))
                        img2 = img2.resize((64,64), Image.ANTIALIAS)
                        images.append(img2)
                        classes.append(breed_idx)
                    
        return images, classes

In [ ]:
random_transforms = [transforms.RandomRotation(degrees=5)]

transform = transforms.Compose([transforms.Resize(args.imsize),
                                transforms.CenterCrop(args.imsize),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomApply(random_transforms, p=0.3),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

train_dataset = DogDataset(dataroot=args.dataroot, transform=transform)
num_of_classes = len(set(train_dataset.classes))

In [ ]:
"""
x = next(iter(train_dataloader))
fig = plt.figure(figsize=(25, 16))
for ii, (img, breed_idx) in enumerate(zip(x[0][:32], x[1][:32])):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    
    img = img.numpy().transpose(1, 2, 0)
    img_class = breeds[breed_idx]
    plt.title(img_class)
    plt.imshow(img)
"""

**Model Definition**

* Weight initialization 

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        xavier_uniform_(m.weight)
        m.bias.data.fill_(0.)

* Customer Layers

In [ ]:
def snconv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
    return spectral_norm(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                   stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias))

In [ ]:
def snlinear(in_features, out_features):
    return spectral_norm(nn.Linear(in_features=in_features, out_features=out_features))

In [ ]:
def sn_embedding(num_embeddings, embedding_dim):
    return spectral_norm(nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim))


In [ ]:
class Self_Attn(nn.Module):
    """ Self attention Layer"""

    def __init__(self, in_channels):
        super(Self_Attn, self).__init__()
        self.in_channels = in_channels
        self.snconv1x1_theta = snconv2d(in_channels=in_channels, out_channels=in_channels//8, kernel_size=1, stride=1, padding=0)
        self.snconv1x1_phi = snconv2d(in_channels=in_channels, out_channels=in_channels//8, kernel_size=1, stride=1, padding=0)
        self.snconv1x1_g = snconv2d(in_channels=in_channels, out_channels=in_channels//2, kernel_size=1, stride=1, padding=0)
        self.snconv1x1_attn = snconv2d(in_channels=in_channels//2, out_channels=in_channels, kernel_size=1, stride=1, padding=0)
        self.maxpool = nn.MaxPool2d(2, stride=2, padding=0)
        self.softmax  = nn.Softmax(dim=-1)
        self.sigma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        """
            inputs :
                x : input feature maps(B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        _, ch, h, w = x.size()
        # Theta path
        theta = self.snconv1x1_theta(x)
        theta = theta.view(-1, ch//8, h*w)
        # Phi path
        phi = self.snconv1x1_phi(x)
        phi = self.maxpool(phi)
        phi = phi.view(-1, ch//8, h*w//4)
        # Attn map
        attn = torch.bmm(theta.permute(0, 2, 1), phi)
        attn = self.softmax(attn)
        # g path
        g = self.snconv1x1_g(x)
        g = self.maxpool(g)
        g = g.view(-1, ch//2, h*w//4)
        # Attn_g
        attn_g = torch.bmm(g, attn.permute(0, 2, 1))
        attn_g = attn_g.view(-1, ch//2, h, w)
        attn_g = self.snconv1x1_attn(attn_g)
        # Out
        out = x + self.sigma*attn_g
        return out

In [ ]:
class ConditionalBatchNorm2d(nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.bn = nn.BatchNorm2d(num_features, momentum=0.001, affine=False)
        self.embed = nn.Embedding(num_classes, num_features * 2)
        self.embed.weight.data[:, :num_features].fill_(1.)  # Initialize scale to 1
        self.embed.weight.data[:, num_features:].zero_()  # Initialize bias at 0

    def forward(self, x, y):
        out = self.bn(x)
        gamma, beta = self.embed(y).chunk(2, 1)
        out = gamma.view(-1, self.num_features, 1, 1) * out + beta.view(-1, self.num_features, 1, 1)
        return out

* Generator Model

In [ ]:
class GenBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_classes):
        super(GenBlock, self).__init__()
        self.cond_bn1 = ConditionalBatchNorm2d(in_channels, num_classes)
        self.relu = nn.ReLU(inplace=True)
        #self.relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.snconv2d1 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.cond_bn2 = ConditionalBatchNorm2d(out_channels, num_classes)
        self.snconv2d2 = snconv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.snconv2d0 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, labels):
        x0 = x

        x = self.cond_bn1(x, labels)
        x = self.relu(x)
        x = F.interpolate(x, scale_factor=2, mode='nearest') # upsample
        #x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True) # upsample
        x = self.snconv2d1(x)
        x = self.cond_bn2(x, labels)
        x = self.relu(x)
        x = self.snconv2d2(x)

        x0 = F.interpolate(x0, scale_factor=2, mode='nearest') # upsample
        #x0 = F.interpolate(x0, scale_factor=2, mode='bilinear', align_corners=True) # upsample
        x0 = self.snconv2d0(x0)

        out = x + x0
        return out

class Generator(nn.Module):
    """Generator."""

    def __init__(self, z_dim, g_conv_dim, num_classes):
        super(Generator, self).__init__()

        self.z_dim = z_dim
        self.g_conv_dim = g_conv_dim

        self.snlinear0 = snlinear(in_features=z_dim, out_features=g_conv_dim*8*4*4)
        
        self.block1 = GenBlock(g_conv_dim*8, g_conv_dim*8, num_classes)
        self.block2 = GenBlock(g_conv_dim*8, g_conv_dim*4, num_classes)
        self.block3 = GenBlock(g_conv_dim*4, g_conv_dim*2, num_classes)
        
        self.self_attn = Self_Attn(g_conv_dim*2)
        
        self.block4 = GenBlock(g_conv_dim*2, g_conv_dim, num_classes)
        
        #self.attn1 = Self_Attn(g_conv_dim*4)
        
        self.bn = nn.BatchNorm2d(g_conv_dim, eps=1e-5, momentum=0.0001, affine=True)
        self.relu = nn.ReLU(inplace=True)
   
        self.snconv2d1 = snconv2d(in_channels=g_conv_dim, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

        # Weight init
        self.apply(init_weights)

    def forward(self, z, labels):
        #z = self.pixelnorm(z)
        
        # n x z_dim -> n x g_conv_dim*8*4*4(8192)
        act0 = self.snlinear0(z)          
        
        # n x g_conv_dim*8*4*4(8192) -> n x g_conv_dim*8 x 4 x 4(n x 512 x 4 x 4)
        act0 = act0.view(-1, self.g_conv_dim*8, 4, 4) 
        
        # n x 512 x 4 x 4 ->  x 512 x 8 x 8
        act1 = self.block1(act0, labels)    
        
        # n x 512 x 8 x 8 -> n x 256 x 16 x 16
        act2 = self.block2(act1, labels)    
        
        #act2 = self.attn1(act2)
        
        # n x 256 x 16 x 16 -> n x 128 x 32 x 32
        act3 = self.block3(act2, labels)    
        
        # n x 128 x 32 x 32 -> n x 128 x 32 x 32
        act3 = self.self_attn(act3)         
        
        # n x 128 x 32 x 32 -> n x 64 x 64 x 64
        act4 = self.block4(act3, labels)    
        
        #act4 = self.attn2(act4)
        #act5 = self.block5(act4, labels)    
        
        act4 = self.bn(act4)                
        act4 = self.relu(act4)              
        
        
        # n x 64 x 64 x 64 -> n x 3 x 64 x 64
        act5 = self.snconv2d1(act4)   
        
        #act5 = self.pixelnorm(act5)
        
        #  n x 3 x 64 x 64
        act5 = self.tanh(act5)              
        return act5

* Discriminator Model

In [ ]:
class DiscOptBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DiscOptBlock, self).__init__()
        self.snconv2d1 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.snconv2d2 = snconv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.downsample = nn.AvgPool2d(2)
        self.snconv2d0 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x0 = x

        x = self.snconv2d1(x)
        x = self.relu(x)
        x = self.snconv2d2(x)
        x = self.downsample(x)

        x0 = self.downsample(x0)
        x0 = self.snconv2d0(x0)

        out = x + x0
        return out


class DiscBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DiscBlock, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.snconv2d1 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.snconv2d2 = snconv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.downsample = nn.AvgPool2d(2)
        self.ch_mismatch = False
        if in_channels != out_channels:
            self.ch_mismatch = True
        self.snconv2d0 = snconv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, downsample=True):
        x0 = x

        x = self.relu(x)
        x = self.snconv2d1(x)
        x = self.relu(x)
        x = self.snconv2d2(x)
        if downsample:
            x = self.downsample(x)

        if downsample or self.ch_mismatch:
            x0 = self.snconv2d0(x0)
            if downsample:
                x0 = self.downsample(x0)

        out = x + x0
        return out


class Discriminator(nn.Module):
    """Discriminator."""

    def __init__(self, d_conv_dim, num_classes):
        super(Discriminator, self).__init__()
        self.d_conv_dim = d_conv_dim
        self.opt_block1 = DiscOptBlock(3, d_conv_dim)
        self.block1 = DiscBlock(d_conv_dim, d_conv_dim*2)
        self.self_attn = Self_Attn(d_conv_dim*2)
        self.block2 = DiscBlock(d_conv_dim*2, d_conv_dim*4)
        self.block3 = DiscBlock(d_conv_dim*4, d_conv_dim*8)
        self.block5 = DiscBlock(d_conv_dim*8, d_conv_dim*8)
        self.relu = nn.ReLU(inplace=True)
        self.snlinear1 = snlinear(in_features=d_conv_dim*8, out_features=1)
        self.sn_embedding1 = sn_embedding(num_classes, d_conv_dim*8)

        # Weight init
        self.apply(init_weights)
        xavier_uniform_(self.sn_embedding1.weight)

    def forward(self, x, labels):
        # x = n x 3 x 64 x 64
        # n x 3 x 64 x 64  ->  n x d_conv_dim   x 32 x 32
        h0 = self.opt_block1(x) 
        
        # n x d_conv_dim x 32 x 32  ->  n x d_conv_dim*2 x 16 x 16
        h1 = self.block1(h0)    
        
        # n x d_conv_dim*2 x 16 x 16    n x d_conv_dim*2 x 16 x 16
        h1 = self.self_attn(h1) 
        
        # n x d_conv_dim*2 x 16 x 16 -> n x d_conv_dim*4 x  8 x  8
        h2 = self.block2(h1)    
        
        # n x d_conv_dim*4 x 8 x 8 -> n x d_conv_dim*8 x 4 x 4
        h3 = self.block3(h2)    
        
        #h4 = self.block4(h3)    # n x d_conv_dim*16 x 4 x  4
        
        # n x d_conv_dim*8 x 4 x 4 -> # n x d_conv_dim*8 x 4 x 4
        h5 = self.block5(h3, downsample=False)  
        h5 = self.relu(h5)   
        
        # n x d_conv_dim*8 x 4 x 4 -> n x d_conv_dim*8
        h6 = torch.sum(h5, dim=[2,3])   
        
        # n x d_conv_dim*8 -> n x 1
        output1 = torch.squeeze(self.snlinear1(h6)) 
        
        # Projection  -> n x d_conv_dim*8
        h_labels = self.sn_embedding1(labels)   
        
        # n x d_conv_dim*8 -> n x d_conv_dim*8
        proj = torch.mul(h6, h_labels)         
        
        # n x d_conv_dim*8 -> n x 1
        output2 = torch.sum(proj, dim=[1])      
        
        # Out: n x 1 
        output = output1 + output2              
        return output

**Training**

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True, num_workers=4)

In [ ]:
device = torch.device('cuda')

### self.build_models()
G = Generator(args.z_dim, args.g_conv_dim, num_of_classes).to(device)
D = Discriminator(args.d_conv_dim, num_of_classes).to(device)

G_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, G.parameters()), args.g_lr, [args.beta1, args.beta2])
D_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), args.d_lr, [args.beta1, args.beta2])

lr_schedulerG = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(G_optimizer, T_0=args.num_epoches//10, eta_min=0.00001)
lr_schedulerD = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(D_optimizer, T_0=args.num_epoches//10, eta_min=0.00005)

In [ ]:
if args.adv_loss == 'dcgan':
    criterion = nn.BCELoss()
    
cudnn.benchmark = True

G.train()
D.train()

label = torch.full((args.batch_size,), 1, device=device)
ones = torch.full((args.batch_size,), 1, device=device)

gen_losses = []
dis_losses = []

# Start training
for epoch in range(args.num_epoches):
        
    for i, (real_images, dog_labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        
        real_images = real_images.to(device)
        dog_labels = torch.tensor(dog_labels, device=device)
                
        # =================================== TRAIN D ============================== #
        G_optimizer.zero_grad()
        D_optimizer.zero_grad()
        
        # TRAIN with REAL        
        # Get D output for real images & real labels
        d_out_real = D(real_images, dog_labels)
        
        
        # Compute D loss with real images & real_labels
        label.fill_(args.real_label_value) +  np.random.uniform(-0.1, 0.1)
        d_loss_real = criterion(torch.sigmoid(d_out_real), label)
        
        # Backward
        d_loss_real.backward()
        
        # Train with FAKE        
        # Create random noise
        z = torch.randn(args.batch_size, args.z_dim, device=device)
        
        # Generate fake images for same dog labels
        fake_images = G(z, dog_labels)
        
        # Get D output for fake images & same dog labels
        d_out_fake = D(fake_images.detach(), dog_labels)
        
        # Compute D loss with fake images & real labels
        label.fill_(args.fake_label_value) + np.random.uniform(0, 0.2)
        d_loss_fake = criterion(torch.sigmoid(d_out_fake), label)
        
        # Backward
        d_loss_fake.backward()
        
        d_loss = d_loss_real + d_loss_fake
        D_optimizer.step()
        
        # ====================================== TRAIN G ============================== #
        G.zero_grad()
        
        # Get D output for fake images & same dog labels
        d_out_fake = D(fake_images, dog_labels)
        
        # Compute G loss with fake images & dog_labels
        label.fill_(args.real_label_value)
        g_loss = criterion(torch.sigmoid(d_out_fake), label)
        
        g_loss.backward()
        
        G_optimizer.step()
        
        gen_losses.append(g_loss.item())
        dis_losses.append(d_loss.item())
                
        lr_schedulerG.step(epoch)
        lr_schedulerD.step(epoch)
    

**Plot Losses**

In [ ]:
# plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(gen_losses,label="G")
plt.plot(dis_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Generate Samples**

In [ ]:
from scipy.stats import truncnorm
def truncated_normal(size, threshold=1):
    values = truncnorm.rvs(-threshold, threshold, size=size)
    return values

def denorm(x):
    out = (x + 1) / 2
    return out.clamp_(0, 1)

*Generate and show samples*

In [ ]:

# show generated samples
z = truncated_normal((args.batch_size, args.z_dim), threshold=1)
gen_z = torch.from_numpy(z).float().to(device)
dog_labels = torch.squeeze(torch.randint(0, num_of_classes, (args.batch_size, ), device=device))
gen_images = denorm(G(gen_z, dog_labels).to("cpu").clone().detach())
gen_images = gen_images.numpy().transpose(0, 2, 3, 1)
fig = plt.figure(figsize=(25, 16))
for ii, img in enumerate(gen_images[:32]):
    ax = fig.add_subplot(4, 8, ii + 1, xticks=[], yticks=[])
    plt.imshow(img)


*Generate Submissions*

In [ ]:
import shutil
def save_images(generator_model, sample_images_path, submission_dir, z_dim, num_images=10000):
    sample_images_dir = Path(sample_images_path)
    sample_images_dir.mkdir(exist_ok=True)
    
    im_batch_size = 50
    device = torch.device('cuda')
    
    for i_batch in range(0, num_images, im_batch_size):
        z = truncated_normal((im_batch_size, z_dim), threshold=1)
        gen_z = torch.from_numpy(z).float().to(device)
        dog_labels = torch.squeeze(torch.randint(0, num_of_classes, (im_batch_size, ), device=device))
        gen_images = generator_model(gen_z, dog_labels)
        images = gen_images.to('cpu').clone().detach()
        images = images.numpy().transpose(0, 2, 3, 1)
        for i_image in range(gen_images.size(0)):
            save_image(denorm(gen_images[i_image, :, :, :]), sample_images_dir / f'image_{i_batch + i_image:05d}.png')
    
    
    submission_dir = Path(submission_dir)
    submission_dir.mkdir(exist_ok=True)
    shutil.make_archive(os.path.join(submission_dir, 'images'), 'zip', sample_images_path)

In [ ]:
save_images(generator_model=G, sample_images_path=args.sample_images_path, submission_dir=args.submission_dir, z_dim=args.z_dim, num_images=10000)

In [ ]:
os.path.exists('../output_images/images.zip')

In [ ]:
os.listdir('./')

In [ ]:
submission_dir = Path(args.submission_dir)
submission_dir.mkdir(exist_ok=True)
shutil.make_archive(os.path.join(args.submission_dir, 'images'), 'zip', args.sample_images_path)

In [ ]:
os.listdir('./submission/')